In [ ]:
import pandas as pd
import psycopg2 as ps
import dotenv
import os
from datetime import datetime
from uuid import uuid4
str(datetime.now()), uuid4().hex

In [ ]:
C = ps.connect(
    host=os.getenv("DB_HOST").strip(),
    port=os.getenv("DB_PORT").strip(),
    user=os.getenv("DB_USER").strip(),
    password=os.getenv("DB_PASSWORD").strip(),
    database=os.getenv("DB_NAME").strip()
)

In [ ]:
pd.read_sql_query("""

with keyevents as (
    select 
        (regexp_matches(source_url, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?([^:\/\n]+)', 'g'))[1] as url,
        *
    from keyevents
)
select 
    url, count(*) 
from keyevents
group by url
order by count(*) desc
""", con=C)

In [ ]:
pd.read_sql_query("""

with keyevents as (
    select 
        * 
    from keyevents limit 10
)
select 
    (regexp_matches(source_url, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?([^:\/\n]+)', 'g'))[1]
from keyevents
""", con=C)

In [ ]:
pd.read_sql_query("""

select 
    * 
from keyevents 
where record_time > NOW() - INTERVAL '1 hour' 
limit 10

""", con=C)

In [ ]:
pd.read_sql_query("""

with last_hour_events as (
    select 
        * 
    from keyevents 
    where record_time > NOW() - INTERVAL '1 hour' 
), word_count as (
    select 
        count(*) as word_count
    from last_hour_events 
    where is_end_of_word is true or is_end_of_line is true 
),
returns_count as (
    select
        count(*) as error_count
    from last_hour_events
    where is_return is true
),
total_count as (
    select
        count(*) as total_count
    from last_hour_events
)
select 
    total_count,
    error_count,
    word_count
from word_count, returns_count, total_count

""", con=C)